In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.7 MB/s eta 0:00:00


#데이터 가져오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
train = pd.read_csv('/content/drive/MyDrive/악성 URL/DATA/train_FE')

# 데이터 전처리(단축 url 사용 및 필요없는 것 제외)

In [ ]:
train_df = train.loc[:,['URL', 'label', 'length', 'subdomain_count', 'special_char_count',
       'number_of_meaning_words', 'tld_malicious', 'path_depth',
       'max_numeric_sequence','https', 'blacklist_word_count',
       'digit_count', 'suspicious_keyword_flag']]

In [ ]:
train_df['tld_malicious'] = train_df['tld_malicious']*1
train_df['tld_malicious'].value_counts()

,count
tld_malicious,
0,6225887
1,769169


#데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split
X,y = train_df.drop(columns=['URL','label']),train_df['label']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#하이퍼 파라미터 튜닝(GridSearchCV)

In [ ]:
from catboost import CatBoostClassifier

cat_boost = CatBoostClassifier(
    thread_count=-1,  # 모든 CPU 코어 사용 (병렬 처리)
    early_stopping_rounds=50,
)

param_grid = {
    'iterations': [500, 1000], #부스팅 트리의 개수
    'learning_rate': [0.01, 0.05, 0.1], #학습률
    'depth': [4, 6, 8], #트리의 깊이
    'l2_leaf_reg': [3, 5, 10], #L2 정규
}

grid_search_result = cat_boost.grid_search(param_grid, X_train[:10000], y_train[:10000])
print('최적의 파라미터 조합:', grid_search_result['params'])


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
425:	learn: 0.2401331	test: 0.2625003	best: 0.2625003 (425)	total: 3.54s	remaining: 4.78s
426:	learn: 0.2400338	test: 0.2624258	best: 0.2624258 (426)	total: 3.55s	remaining: 4.77s
427:	learn: 0.2399797	test: 0.2623984	best: 0.2623984 (427)	total: 3.56s	remaining: 4.76s
428:	learn: 0.2399133	test: 0.2623830	best: 0.2623830 (428)	total: 3.57s	remaining: 4.75s
429:	learn: 0.2398803	test: 0.2623613	best: 0.2623613 (429)	total: 3.58s	remaining: 4.74s
430:	learn: 0.2398477	test: 0.2623461	best: 0.2623461 (430)	total: 3.58s	remaining: 4.73s
431:	learn: 0.2397506	test: 0.2623079	best: 0.2623079 (431)	total: 3.59s	remaining: 4.72s
432:	learn: 0.2396899	test: 0.2622770	best: 0.2622770 (432)	total: 3.6s	remaining: 4.72s
433:	learn: 0.2396125	test: 0.2622424	best: 0.2622424 (433)	total: 3.61s	remaining: 4.71s
434:	learn: 0.2395512	test: 0.2621868	best: 0.2621868 (434)	total: 3.62s	remaining: 4.7s
435:	learn: 0.2395181	test: 0.2621743	best: 0.2621743 (435)	total:

In [ ]:
best_params = grid_search_result['params']
best_params

# Catboost 모델 불러오기

In [ ]:
# grid_search_result에서 최적의 파라미터 조합을 가져옴
best_params = grid_search_result['params']

# 최적의 파라미터로 모델을 생성하고 학습
cat_model = CatBoostClassifier(iterations=best_params['iterations'],
                                learning_rate=best_params['learning_rate'],
                                depth=best_params['depth'],
                                l2_leaf_reg=best_params['l2_leaf_reg'])

# 모델 학습
cat_model.fit(X_train, y_train)

# 예측
y_pred = cat_model.predict(X_test)



0:	learn: 0.6380631	total: 3.93s	remaining: 32m 39s
1:	learn: 0.5867980	total: 6.39s	remaining: 26m 32s
2:	learn: 0.5447491	total: 7.92s	remaining: 21m 52s
3:	learn: 0.5110639	total: 9.52s	remaining: 19m 40s
4:	learn: 0.4798610	total: 11.1s	remaining: 18m 20s
5:	learn: 0.4542529	total: 12.7s	remaining: 17m 25s
6:	learn: 0.4305703	total: 14.3s	remaining: 16m 47s
7:	learn: 0.4094812	total: 15.9s	remaining: 16m 17s
8:	learn: 0.3935355	total: 18.7s	remaining: 17m
9:	learn: 0.3774801	total: 21.6s	remaining: 17m 36s
10:	learn: 0.3651293	total: 23.8s	remaining: 17m 36s
11:	learn: 0.3537248	total: 25.4s	remaining: 17m 11s
12:	learn: 0.3434909	total: 27s	remaining: 16m 51s
13:	learn: 0.3347325	total: 28.6s	remaining: 16m 32s
14:	learn: 0.3272955	total: 30.1s	remaining: 16m 14s
15:	learn: 0.3208205	total: 32.1s	remaining: 16m 9s
16:	learn: 0.3145794	total: 34.9s	remaining: 16m 31s
17:	learn: 0.3087322	total: 37.2s	remaining: 16m 35s
18:	learn: 0.3047123	total: 38.7s	remaining: 16m 20s
19:	learn:

#ROC AUC로 평가하기

In [ ]:
from sklearn.metrics import roc_auc_score

y_pred_proba = cat_model.predict_proba(X_test)[:,1]
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f'ROC AUC Score: {roc_auc}')

ROC AUC Score: 0.9294462725266183
